In [ ]:
import requests
from time import sleep

In [ ]:
url = 'URL-OF-API'
headers = {'Api-Token': "XXXXXXXXXXXXXXXXXXXXX"}

# Create an Entry to the Catalog

In [ ]:
data = {"package_metadata": {"title": "Test workflow3",
                             "notes": "This workflow performs entity matching",
                             "tags": ["STELAR", "Entity matching", "Entity resolution"]}}
response = requests.post(url + 'workflow/publish', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
package_id = response.json()['result']['package_id']

# Create a workflow

In [ ]:
data = {"tags": {}}
response = requests.post(url + 'workflow/execution/create', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
workflow_exec_id = response.json()['workflow_exec_id']

# Create a task

In [ ]:
data = {"workflow_exec_id": workflow_exec_id,
        "docker_image": "alzeakis/pytokenjoin:v3",
        "input": [  "0059004a-67b8-4445-b9d6-5b0475784c49",
                    "2350a24b-87af-4505-aafb-72a15e0c118c"],
        "parameters": {
            "col_id_left": 1,
            "col_text_left": 2,
            "separator_left": " ",
            "col_id_right": 0,
            "col_text_right": 1,
            "separator_right": " ",
            "k": 1,
            "delta_alg": "1",
            "output_file": "out.csv",
            "method": "knn",
            "similarity": "jaccard",
            "foreign": "foreign"
        },
        "tags": {}}
response = requests.post(url + 'task/execution/create', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
task_exec_id = response.json()['task_exec_id']

# Track task metadata

In [ ]:
state = 'running'
data = {"task_exec_id": task_exec_id,
        "package_id": ""}
while state != 'succeeded' and state != 'failed':
    response = requests.post(url + 'task/execution/track', 
                          json=data, headers=headers)
    print(response.status_code)
    print(response.json())   
    state = response.json()['metadata']['state']
    sleep(2)

# Commit workflow metadata

In [ ]:
data = {"workflow_exec_id": workflow_exec_id, "state": "succeeded"}
response = requests.post(url + 'workflow/execution/commit', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())  

# Get workflow metadata

In [ ]:
response = requests.get(url + 'workflow/execution/read?id=' + workflow_exec_id, 
                        headers=headers)
print(response.status_code)
print(response.json())  